In [8]:
!pip install --upgrade transformers -q
!pip install accelerate -q

# GPTQ Dependencies
!pip install --upgrade optimum -q
!pip install --upgrade auto-gptq -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
optimum 1.19.0 requires transformers[sentencepiece]<4.40.0,>=4.26.0, but you have transformers 4.40.0 which is incompatible.


In [9]:
pip install accelerate

In [10]:
!pip install pyngrok

In [11]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

https://svb7b511e89-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [12]:
from flask import Flask, request, jsonify
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import json
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:",device)

# Initialize GPT model and tokenizer
model_id = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    padding=True,
    pad_side="left",
    use_fast=True
)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=False,
    revision="main"
)

# Create a pipeline
pipe = pipeline(
    task='text-generation',
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    do_sample=True, # creative generation by discouraging greedy decoding
    temperature=0.75,
    top_p=0.95,
    return_full_text=False  # Only return the current output instead of returning complete prompt
)

def parse_first_json(json_str):
    end_index = json_str.find("}") # Find the index of the first closing curly brace
    if end_index != -1:  # If a closing brace is found
        first_json_obj = json_str[:end_index + 1] # Parse the substring containing the first JSON object
        parsed_json = json.loads(first_json_obj) # Parse the JSON object
        return parsed_json
    else:
        return None



device: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4225: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [13]:
from flask import Flask, jsonify, request
from pyngrok import ngrok
import os
import threading

os.environ["FLASK_DEBUG"] = "development"
app = Flask(__name__)
port = 5000
ngrok.set_auth_token("2WwIjT7OELQ8vcfHm7zFZJ9lYV6_58wAxRAfq2MpTJFoFCmtw")
public_url = ngrok.connect(port).public_url
print(public_url)
app.config["BASE_URL"] = public_url
@app.route('/generate', methods=['POST'])
def home():
            # Retrieve data from the request
    data = request.json
    story=data["story"]
    user_input=data["input"]
    system = f"""
    Embark on a journey as the virtuoso architect of interactive fiction, celebrated for crafting mesmerizing tales. Your quest: expand the narrative provided below in square brackets, 
    sculpting a storyline no longer than 50 words. Offer three captivating choices, each laden with unique consequences and a measure of destruction points. 
    Present them within a JSON object featuring "story," "options," and "destruction_points" keys.

    If the user seeks enlightenment, be prepared to unveil detailed explanations for each scenario, guiding them through the intricacies of your narrative world.
    
    Before you, lies the vast canvas of imagination. Paint your tale with finesse, for with each brushstroke, the destiny of your narrative universe unfolds.

    {{
    "story": string, (Continue the narrative from the provided story)
    "options": list, (Present three distinct choices)
    "destruction_points": list, (Assign destruction points to each option)
    }}

    Delve deep, weave your magic, and let your creativity illuminate the path to storytelling greatness!
    Output must be in JSON format.
    """


    assistant = """
    {"story": "As you flee from the cursed room, you hear the mummy's voice growing louder. Sweat pouring down your face, you make a hasty decision.", "options": ["Seek help from the village elder.", "Search for a way to break the curse.", "Abandon your quest and leave the relics behind."],"destruction_points": ["0", "-5", "-10"]}
    """

    user = f"""
    Start a new story if the story given below in the square brackets is empty, otherwise continue the story so far following the rules provided earlier.


    [{story}]
    """

    # Create a chat
    chat = [
        {"role": "user", "content": system},
        {"role": "assistant", "content": assistant},
        {"role": "user", "content": user}
    ]

    # Prepare the prompt using the chat template
    prompt = tokenizer.apply_chat_template(chat, tokenize=False)
    # Run the pipeline to generate the model output
    outputs = pipe(prompt)
    output = outputs[0]["generated_text"].strip()

    # Sometimes the model generates notes/explanation after JSON, so we only parse the first JSON object
    data = parse_first_json(output)
    narrative = data["story"]
    options = data["options"]
    points=data["destruction_points"]
    print(narrative)
    [print(opt) for opt in options]

    story += user_input





    return jsonify({"story": narrative, "options": options,"points":points})

threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()
# if __name__ == '__main__':
#     app.run()

https://b940-34-32-130-153.ngrok-free.app
